In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from statsmodels.distributions.empirical_distribution import ECDF
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

In [2]:
cd ..

C:\Users\Elena\PycharmProjects\recsys


In [4]:
path = 'data/ml-1m/'

In [5]:
users = pd.read_csv(
    path+"users.dat",
    sep="::",
    names=["user_id", "sex", "age_group", "occupation", "zip_code"],
    encoding='latin-1',
    engine='python'
)

ratings = pd.read_csv(
    path+"ratings.dat",
    sep="::",
    names=["user_id", "movie_id", "rating", "unix_timestamp"],
    encoding='latin-1',
    engine='python'
)

movies = pd.read_csv(
    path+"movies.dat", sep="::", names=["movie_id", "title", "genres"],
    encoding='latin-1',
    engine='python'
)

In [6]:
users[:2]

,user_id,sex,age_group,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072


In [7]:
ratings[:2]

,user_id,movie_id,rating,unix_timestamp
0,1,1193,5,978300760
1,1,661,3,978302109


In [8]:
movies[:2]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy


In [9]:
ratings_df = pd.merge(ratings, movies)[['user_id', 'title', 'rating', 'unix_timestamp']]
ratings_df["user_id"] = ratings_df["user_id"].astype(str)

In [10]:
ratings_per_user = ratings_df.groupby('user_id').rating.count()
ratings_per_item = ratings_df.groupby('title').rating.count()

print(f"Total No. of users: {len(ratings_df.user_id.unique())}")
print(f"Total No. of items: {len(ratings_df.title.unique())}")
print("\n")

print(f"Max observed rating: {ratings_df.rating.max()}")
print(f"Min observed rating: {ratings_df.rating.min()}")
print("\n")

print(f"Max no. of user ratings: {ratings_per_user.max()}")
print(f"Min no. of user ratings: {ratings_per_user.min()}")
print(f"Median no. of ratings per user: {ratings_per_user.median()}")
print("\n")

print(f"Max no. of item ratings: {ratings_per_item.max()}")
print(f"Min no. of item ratings: {ratings_per_item.min()}")
print(f"Median no. of ratings per item: {ratings_per_item.median()}")

Total No. of users: 6040
Total No. of items: 3706


Max observed rating: 5
Min observed rating: 1


Max no. of user ratings: 2314
Min no. of user ratings: 20
Median no. of ratings per user: 96.0


Max no. of item ratings: 3428
Min no. of item ratings: 1
Median no. of ratings per item: 123.5


In [11]:
# Определим функцию, которая будет получать n оценок для каждого пользователя:
def get_last_n_ratings_by_user(
    df, n, min_ratings_per_user=1, user_colname="user_id", timestamp_colname="unix_timestamp"
):
    return (
        df.groupby(user_colname)
        .filter(lambda x: len(x) >= min_ratings_per_user)
        .sort_values(timestamp_colname)
        .groupby(user_colname)
        .tail(n)
        .sort_values(user_colname)
    )

In [12]:
get_last_n_ratings_by_user(ratings_df, 1)

,user_id,title,rating,unix_timestamp
28501,1,Pocahontas (1995),5,978824351
482398,10,Hero (1992),5,980638688
800008,100,Apocalypse Now (1979),2,977594963
496041,1000,"Streetcar Named Desire, A (1951)",5,975042421
305563,1001,Austin Powers: The Spy Who Shagged Me (1999),2,1028605534
...,...,...,...,...
767773,995,French Kiss (1995),3,975099776
573889,996,Almost Famous (2000),5,1001227064
76463,997,Gladiator (2000),4,978915132
998801,998,See the Sea (Regarde la mer) (1997),5,975192573


In [13]:
# функция, которая будет помечать n оценок на пользователя так, чтобы они 
# попадали бы в проверочную выборку. Делается это с использованием столбца is_valid:
def mark_last_n_ratings_as_validation_set(
    df, n, min_ratings=1, user_colname="user_id", timestamp_colname="unix_timestamp"
):
    """
    Отмечает n последних по времени оценок, что включает их в проверочную выборку.
    Делается это путём добавления дополнительного столбца 'is_valid' в df.
    :param df: объект DataFrame, содержащий оценки, данные пользователем
    :param n: количество оценок, которые надо включить в проверочную выборку
    :param min_ratings: включать лишь пользователей, имеющих более этого количества оценок
    :param user_id_colname: имя столбца, содержащего идентификатор пользователя
    :param timestamp_colname: имя столбца, содержащего отметку времени
    :return: тот же df, в который добавлен дополнительный столбец 'is_valid'
    """
    df["is_valid"] = False
    df.loc[
        get_last_n_ratings_by_user(
            df,
            n,
            min_ratings,
            user_colname=user_colname,
            timestamp_colname=timestamp_colname,
        ).index,
        "is_valid",
    ] = True
    return df

In [14]:
mark_last_n_ratings_as_validation_set(ratings_df, 1)

,user_id,title,rating,unix_timestamp,is_valid
0,1,One Flew Over the Cuckoo's Nest (1975),5,978300760,False
1,2,One Flew Over the Cuckoo's Nest (1975),5,978298413,False
2,12,One Flew Over the Cuckoo's Nest (1975),4,978220179,False
3,15,One Flew Over the Cuckoo's Nest (1975),4,978199279,False
4,17,One Flew Over the Cuckoo's Nest (1975),5,978158471,False
...,...,...,...,...,...
1000204,5949,Modulations (1998),5,958846401,False
1000205,5675,Broken Vessels (1998),3,976029116,False
1000206,5780,White Boys (1999),1,958153068,False
1000207,5851,One Little Indian (1973),5,957756608,False


In [15]:
train_df = ratings_df[ratings_df.is_valid==False]
valid_df = ratings_df[ratings_df.is_valid==True]

In [16]:
median_rating = train_df.rating.median(); median_rating

4.0

In [17]:
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error

predictions = np.array([median_rating]* len(valid_df))

mae = mean_absolute_error(valid_df.rating, predictions)
mse = mean_squared_error(valid_df.rating, predictions)
rmse = math.sqrt(mse)

print(f'mae: {mae}')
print(f'mse: {mse}')
print(f'rmse: {rmse}')

mae: 0.91158940397351
mse: 1.5304635761589405
rmse: 1.2371190630488806


In [18]:
# Кодируем юзеров и фильмы
user_lookup = {v: i+1 for i, v in enumerate(ratings_df['user_id'].unique())}
movie_lookup = {v: i+1 for i, v in enumerate(ratings_df['title'].unique())}

In [19]:
class UserItemRatingDataset(Dataset):
    def __init__(self, df, movie_lookup, user_lookup):
        self.df = df
        self.movie_lookup = movie_lookup
        self.user_lookup = user_lookup

    def __getitem__(self, index):
        row = self.df.iloc[index]
        user_id = self.user_lookup[row.user_id]
        movie_id = self.movie_lookup[row.title]
        
        rating = torch.tensor(row.rating, dtype=torch.float32)
        
        return (user_id, movie_id), rating

    def __len__(self):
        return len(self.df)

In [20]:
train_dataset = UserItemRatingDataset(train_df, movie_lookup, user_lookup)
valid_dataset = UserItemRatingDataset(valid_df, movie_lookup, user_lookup)

In [21]:
import torch
from torch import nn

# Определим модель
class MfDotBias(nn.Module):

    def __init__(
        self, n_factors, n_users, n_items, ratings_range=None, use_biases=True
    ):
        super().__init__()
        self.bias = use_biases
        self.y_range = ratings_range
        self.user_embedding = nn.Embedding(n_users+1, n_factors, padding_idx=0)
        self.item_embedding = nn.Embedding(n_items+1, n_factors, padding_idx=0)

        if use_biases:
            self.user_bias = nn.Embedding(n_users+1, 1, padding_idx=0)
            self.item_bias = nn.Embedding(n_items+1, 1, padding_idx=0)

    def forward(self, inputs):
        users, items = inputs
        dot = self.user_embedding(users) * self.item_embedding(items)
        result = dot.sum(1)
        if self.bias:
            result = (
                result + self.user_bias(users).squeeze() + self.item_bias(items).squeeze()
            )

        if self.y_range is None:
            return result
        else:
            return (
                torch.sigmoid(result) * (self.y_range[1] - self.y_range[0])
                + self.y_range[0]
            )

In [23]:
from functools import partial
# !pip install pytorch_accelerated
# !pip install torchmetrics
from pytorch_accelerated import Trainer, notebook_launcher 
from pytorch_accelerated.trainer import TrainerPlaceholderValues, DEFAULT_CALLBACKS
from pytorch_accelerated.callbacks import EarlyStoppingCallback, SaveBestModelCallback, TrainerCallback, StopTrainingError
import torchmetrics

In [24]:
class RecommenderMetricsCallback(TrainerCallback):
    def __init__(self):
        self.metrics = torchmetrics.MetricCollection(
            {
                "mse": torchmetrics.MeanSquaredError(),
                "mae": torchmetrics.MeanAbsoluteError(),
            }
        )

    def _move_to_device(self, trainer):
        self.metrics.to(trainer.device)

    def on_training_run_start(self, trainer, **kwargs):
        self._move_to_device(trainer)

    def on_evaluation_run_start(self, trainer, **kwargs):
        self._move_to_device(trainer)

    def on_eval_step_end(self, trainer, batch, batch_output, **kwargs):
        preds = batch_output["model_outputs"]
        self.metrics.update(preds, batch[1])

    def on_eval_epoch_end(self, trainer, **kwargs):
        metrics = self.metrics.compute()
        
        mse = metrics["mse"].cpu()
        trainer.run_history.update_metric("mae", metrics["mae"].cpu())
        trainer.run_history.update_metric("mse", mse)
        trainer.run_history.update_metric("rmse",  math.sqrt(mse))

        self.metrics.reset()

In [23]:
def train_mf_model():
    model = MfDotBias(
        120, len(user_lookup), len(movie_lookup), ratings_range=[0.5, 5.5]
    )
    loss_func = torch.nn.MSELoss()

    optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)

    create_sched_fn = partial(
        torch.optim.lr_scheduler.OneCycleLR,
        max_lr=0.01,
        epochs=TrainerPlaceholderValues.NUM_EPOCHS,
        steps_per_epoch=TrainerPlaceholderValues.NUM_UPDATE_STEPS_PER_EPOCH,
    )

    trainer = Trainer(
        model=model,
        loss_func=loss_func,
        optimizer=optimizer,
        callbacks=(
            RecommenderMetricsCallback,
            *DEFAULT_CALLBACKS,
            SaveBestModelCallback(watch_metric="mae"),
            EarlyStoppingCallback(
                early_stopping_patience=2,
                early_stopping_threshold=0.001,
                watch_metric="mae",
            ),
        ),
    )

    trainer.train(
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        num_epochs=30,
        per_device_batch_size=512,
        create_scheduler_fn=create_sched_fn,
    )

In [24]:
notebook_launcher(train_mf_model, num_processes=2)

Launching training on one GPU.

Starting training run

Starting epoch 1


100%|██████████| 1576/1576 [03:13<00:00,  8.14it/s]



train_loss_epoch: 6.889003145034498


100%|██████████| 10/10 [00:01<00:00,  8.31it/s]



rmse: 2.649046431224167

mse: 7.017446994781494

mae: 2.2942216396331787

eval_loss_epoch: 7.017446929433151

Starting epoch 2


...


train_loss_epoch: 5.522534759300887


100%|██████████| 10/10 [00:01<00:00,  7.72it/s]



rmse: 2.5081729808089706

mse: 6.290931701660156

mae: 2.1526763439178467

eval_loss_epoch: 6.290932070824407

Improvement of 0.07907247543334961 observed, resetting counter. 
Early stopping counter: 0/2

Starting epoch 5


 52%|█████▏    | 826/1576 [01:34<01:21,  9.25it/s]

KeyboardInterrupt: ignored

In [25]:
ratings_df

,user_id,title,rating,unix_timestamp,is_valid
0,1,One Flew Over the Cuckoo's Nest (1975),5,978300760,False
1,2,One Flew Over the Cuckoo's Nest (1975),5,978298413,False
2,12,One Flew Over the Cuckoo's Nest (1975),4,978220179,False
3,15,One Flew Over the Cuckoo's Nest (1975),4,978199279,False
4,17,One Flew Over the Cuckoo's Nest (1975),5,978158471,False
...,...,...,...,...,...
1000204,5949,Modulations (1998),5,958846401,False
1000205,5675,Broken Vessels (1998),3,976029116,False
1000206,5780,White Boys (1999),1,958153068,False
1000207,5851,One Little Indian (1973),5,957756608,False


In [26]:
# Использование трансформеров для выдачи прогнозов, основанных на последовательностях оценок:

In [27]:
grouped_ratings = ratings_df.sort_values(by='unix_timestamp').groupby('user_id').agg(tuple).reset_index()
grouped_ratings['num_ratings'] = grouped_ratings['rating'].apply(lambda row: len(row))

In [28]:
grouped_ratings[:3]

,user_id,title,rating,unix_timestamp,is_valid,num_ratings
0,1,"(Girl, Interrupted (1999), Cinderella (1950), ...","(4, 5, 4, 5, 3, 5, 4, 4, 5, 4, 5, 3, 4, 4, 4, ...","(978300019, 978300055, 978300055, 978300055, 9...","(False, False, False, False, False, False, Fal...",53
1,10,"(Godfather, The (1972), Pretty Woman (1990), S...","(3, 4, 3, 4, 4, 3, 5, 5, 5, 3, 3, 4, 5, 4, 4, ...","(978224375, 978224375, 978224375, 978224400, 9...","(False, False, False, False, False, False, Fal...",401
2,100,"(Starship Troopers (1997), Star Wars: Episode ...","(3, 4, 4, 3, 4, 3, 1, 1, 5, 4, 4, 3, 4, 2, 3, ...","(977593595, 977593595, 977593607, 977593624, 9...","(False, False, False, False, False, False, Fal...",76


In [29]:
sequence_length = 10

In [30]:
def create_sequences(values, sequence_length):
    sequences = []
    for i, v in enumerate(values):
        seq = values[:i+1]
        if len(seq) > sequence_length:
            seq = seq[i-sequence_length+1:i+1]
        elif len(seq) < sequence_length:
            seq =(*(['[PAD]'] * (sequence_length - len(seq))), *seq)
       
        sequences.append(seq)
    return sequences

In [31]:
# Применение ко всему df
grouped_cols = ['title', 'rating', 'unix_timestamp', 'is_valid'] 
for col in grouped_cols:
    grouped_ratings[col] = grouped_ratings[col].apply(lambda x: create_sequences(x, sequence_length))

In [32]:
# Сейчас у нас имеется одна строка, содержащая все последовательности для конкретного пользователя. 
# надо преобразовать данные таким образом, чтобы каждая последовательность располагалась бы в 
# собственной строке, но, в то же время, сохраняла бы связь с ID пользователя
exploded_ratings = grouped_ratings[['user_id', 'title']].explode('title', ignore_index=True)
dfs = [grouped_ratings[[col]].explode(col, ignore_index=True) for col in grouped_cols[1:]]
seq_df = pd.concat([exploded_ratings, *dfs], axis=1)

In [33]:
seq_df

,user_id,title,rating,unix_timestamp,is_valid
0,1,"([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."
1,1,"([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."
2,1,"([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."
3,1,"([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], Gir...","([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], 4, ...","([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], 978...","([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], Fal..."
4,1,"([PAD], [PAD], [PAD], [PAD], [PAD], Girl, Inte...","([PAD], [PAD], [PAD], [PAD], [PAD], 4, 5, 4, 5...","([PAD], [PAD], [PAD], [PAD], [PAD], 978300019,...","([PAD], [PAD], [PAD], [PAD], [PAD], False, Fal..."
...,...,...,...,...,...
1000204,999,"(General's Daughter, The (1999), Powder (1995)...","(3, 3, 2, 1, 3, 2, 3, 2, 4, 3)","(975364681, 975364717, 975364717, 975364717, 9...","(False, False, False, False, False, False, Fal..."
1000205,999,"(Powder (1995), We're No Angels (1989), Out of...","(3, 2, 1, 3, 2, 3, 2, 4, 3, 3)","(975364717, 975364717, 975364717, 975364743, 9...","(False, False, False, False, False, False, Fal..."
1000206,999,"(We're No Angels (1989), Out of Africa (1985),...","(2, 1, 3, 2, 3, 2, 4, 3, 3, 3)","(975364717, 975364717, 975364743, 975364743, 9...","(False, False, False, False, False, False, Fal..."
1000207,999,"(Out of Africa (1985), Instinct (1999), Corrup...","(1, 3, 2, 3, 2, 4, 3, 3, 3, 2)","(975364717, 975364743, 975364743, 975364784, 9...","(False, False, False, False, False, False, Fal..."


In [34]:
# Для "is_valid" оставляем только последнее значение, для отбора примеров для прогноза
def get_last_entry(sequence):
    return sequence[-1]

seq_df['is_valid'] = seq_df['is_valid'].apply(get_last_entry)

In [35]:
#чтобы облегчить доступ к оценке, которую мы пытаемся спрогнозировать, выделим её в отдельный столбец:
seq_df['target_rating'] = seq_df['rating'].apply(get_last_entry)
seq_df['previous_ratings'] = seq_df['rating'].apply(lambda seq: seq[:-1])
seq_df.drop(columns=['rating'], inplace=True)

In [36]:
# Чтобы модель, при вычислении показателей внутреннего внимания, не использовала бы вспомогательные токены,
# мы можем предоставить трансформеру маску для механизма внутреннего внимания. Маска должна содержать значение 
# True для вспомогательного токена, а для других данных — False. Построим маску для каждой из строк, а так же 
# создадим столбец, содержащий сведения о количестве вспомогательных токенов:

seq_df['pad_mask'] = seq_df['title'].apply(lambda x: (np.array(x) == '[PAD]'))
seq_df['num_pads'] = seq_df['pad_mask'].apply(sum)
seq_df['pad_mask'] = seq_df['pad_mask'].apply(lambda x: x.tolist())

In [37]:
seq_df[:5]

,user_id,title,unix_timestamp,is_valid,target_rating,previous_ratings,pad_mask,num_pads
0,1,"([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",False,4,"([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[True, True, True, True, True, True, True, Tru...",9
1,1,"([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",False,5,"([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[True, True, True, True, True, True, True, Tru...",8
2,1,"([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",False,4,"([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[True, True, True, True, True, True, True, Fal...",7
3,1,"([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], Gir...","([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], 978...",False,5,"([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], 4, ...","[True, True, True, True, True, True, False, Fa...",6
4,1,"([PAD], [PAD], [PAD], [PAD], [PAD], Girl, Inte...","([PAD], [PAD], [PAD], [PAD], [PAD], 978300019,...",False,3,"([PAD], [PAD], [PAD], [PAD], [PAD], 4, 5, 4, 5)","[True, True, True, True, True, False, False, F...",5


In [38]:
train_seq_df = seq_df[seq_df.is_valid == False]
valid_seq_df = seq_df[seq_df.is_valid == True]

In [39]:
#прежде чем мы сможем передать эти данные модели, нужно создать поисковую таблицу для кодирования сведений о 
# пользователях и фильмах. Но в этот раз, при создании поисковой таблицы фильмов, нам нужно учесть наличие в 
# данных вспомогательного токена:
user_lookup = {v: i+1 for i, v in enumerate(ratings_df['user_id'].unique())}
def create_feature_lookup(df, feature):
    lookup = {v: i+1 for i, v in enumerate(df[feature].unique())}
    lookup['[PAD]'] = 0
    return lookup
  
movie_lookup = create_feature_lookup(ratings_df, 'title')

In [40]:
seq_df[:1]

,user_id,title,unix_timestamp,is_valid,target_rating,previous_ratings,pad_mask,num_pads
0,1,"([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...",False,4,"([PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[True, True, True, True, True, True, True, Tru...",9


In [41]:
# Теперь мы работаем с последовательностями оценок, а не с отдельными оценками. 
# Поэтому нужно создать новый класс Dataset, содержащий обработанный DataFrame:
class MovieSequenceDataset(Dataset):
    def __init__(self, df, movie_lookup, user_lookup):
        super().__init__()
        self.df = df
        self.movie_lookup = movie_lookup
        self.user_lookup = user_lookup

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        data = self.df.iloc[index]
        user_id = self.user_lookup[str(data.user_id)]
        movie_ids = torch.tensor([self.movie_lookup[title] for title in data.title])

        previous_ratings = torch.tensor(
            [rating if rating != "[PAD]" else 0 for rating in data.previous_ratings]
        )

        attention_mask = torch.tensor(data.pad_mask)
        target_rating = data.target_rating
        encoded_features = {
            "user_id": user_id,
            "movie_ids": movie_ids,
            "ratings": previous_ratings,
        }

        return (encoded_features, attention_mask), torch.tensor(
            target_rating, dtype=torch.float32
        )

In [42]:
train_dataset = MovieSequenceDataset(train_seq_df, movie_lookup, user_lookup)
valid_dataset = MovieSequenceDataset(valid_seq_df, movie_lookup, user_lookup)

In [57]:
# for features, mask in train_dataset:
#     features, mask

features

({'user_id': 2437,
  'movie_ids': tensor([  52,  247,   24,  516,  536, 2839,  245,   30,   41,  185]),
  'ratings': tensor([4, 5, 4, 5, 4, 2, 4, 4, 4])},
 tensor([False, False, False, False, False, False, False, False, False, False]))

In [58]:
movies_num_unique = len(movie_lookup)
embedding_size = 120
movies_embeddings = nn.Embedding(movies_num_unique + 1, embedding_size, padding_idx=0)
encoded_movies = movies_embeddings(features["movie_ids"])

TypeError: tuple indices must be integers or slices, not str

In [47]:
sequence_length=10
position_embeddings = nn.Embedding(sequence_length, 120)
positions = torch.arange( 0, 10, 1, dtype=int, ) #device=features["movie_ids"].device)        )     
position_embeddings(positions)[1]

tensor([-0.0127,  0.2408,  0.1325,  0.7642,  1.0950,  0.3399,  0.7200,  0.4114,
         1.9312,  1.0119, -1.4364, -1.1299, -0.1360,  1.6354,  0.6547,  0.5760,
         1.1415,  0.0186, -1.8058,  0.9254, -0.3753,  1.0331, -0.6867,  0.6368,
        -0.9727,  0.9585,  1.6192,  1.4506,  0.2695, -0.2104, -0.7328,  0.1043,
         0.3488,  0.9676, -0.4657,  1.6048, -2.4801, -0.4175, -1.1955,  0.8123,
        -1.9006,  0.2286,  0.0249, -0.3460,  0.2868, -0.7308,  0.1748, -1.0939,
        -1.6022,  1.3529,  1.2888,  0.0523, -1.5469,  0.7567,  0.7755,  2.0265,
         0.0358,  0.1206, -0.8057, -0.2076, -0.9319, -1.5910, -1.1360, -0.5226,
        -0.5188, -1.5013, -1.9267,  0.1279,  1.0229, -0.5558,  0.7043,  0.7099,
         1.7744, -0.9216,  0.9624, -0.3370, -1.1753,  0.3581,  0.4788,  1.3537,
         0.5261,  2.1120, -0.5208, -0.9320,  0.1852,  1.0687,  1.3065,  0.4598,
        -0.8146, -1.0212, -0.4949, -0.5923,  0.1543,  0.4408, -0.1483, -2.3184,
        -0.3980,  1.0805, -1.7809,  1.50

In [59]:
# Опишем модель, основанную на трансформере. Для начала, учитывая то, что модель матричной факторизации способна 
# достичь хороших результатов, используя лишь идентификаторы пользователя и фильма, пока включим сюда лишь эти данные:
class BstTransformer(nn.Module):
    def init(
        self,
        movies_num_unique,
        users_num_unique,
        sequence_length=10,
        embedding_size=120,
        num_transformer_layers=1,
        ratings_range=(0.5, 5.5),
    ):
        super().init()
        self.sequence_length = sequence_length
        self.y_range = ratings_range
        self.movies_embeddings = nn.Embedding(
            movies_num_unique + 1, embedding_size, padding_idx=0
        )
        self.user_embeddings = nn.Embedding(users_num_unique + 1, embedding_size)
        self.position_embeddings = nn.Embedding(sequence_length, embedding_size)
        self.encoder = nn.TransformerEncoder(
            encoder_layer=nn.TransformerEncoderLayer(
                d_model=embedding_size,
                nhead=12,
                dropout=0.1,
                batch_first=True,
                activation="gelu",
            ),
            num_layers=num_transformer_layers,
        )
        self.linear = nn.Sequential(
            nn.Linear(
                embedding_size + (embedding_size * sequence_length),
                1024,
            ),
            nn.BatchNorm1d(1024),
            nn.Mish(),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.Mish(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.Mish(),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )
    def forward(self, inputs):
        features, mask = inputs
        encoded_user_id = self.user_embeddings(features["user_id"])
        user_features = encoded_user_id
        encoded_movies = self.movies_embeddings(features["movie_ids"])
        positions = torch.arange(
            0, self.sequence_length, 1, dtype=int, device=features["movie_ids"].device
        )
        positions = self.position_embeddings(positions)
        transformer_features = encoded_movies + positions
        transformer_output = self.encoder(
            transformer_features, src_key_padding_mask=mask
        )
        transformer_output = torch.flatten(transformer_output, start_dim=1)
        combined_output = torch.cat((transformer_output, user_features), dim=1)
        rating = self.linear(combined_output)
        rating = rating.squeeze()
        if self.y_range is None:
            return rating
        else:
            return rating * (self.y_range[1] - self.y_range[0]) + self.y_range[0]

In [60]:
def train_seq_model():
    model = BstTransformer(
        len(movie_lookup), len(user_lookup), sequence_length , embedding_size=120
    )
    loss_func = torch.nn.MSELoss()

    optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)

    create_sched_fn = partial(
        torch.optim.lr_scheduler.OneCycleLR,
        max_lr=0.01,
        epochs=TrainerPlaceholderValues.NUM_EPOCHS,
        steps_per_epoch=TrainerPlaceholderValues.NUM_UPDATE_STEPS_PER_EPOCH,
    )

    trainer = Trainer(
        model=model,
        loss_func=loss_func,
        optimizer=optimizer,
        callbacks=(
            RecommenderMetricsCallback,
            *DEFAULT_CALLBACKS,
            SaveBestModelCallback(watch_metric="mae"),
            EarlyStoppingCallback(
                early_stopping_patience=2,
                early_stopping_threshold=0.001,
                watch_metric="mae",
            ),
        ),
    )

    trainer.train(
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        num_epochs=10,
        per_device_batch_size=512,
        create_scheduler_fn=create_sched_fn,
    )

In [61]:
notebook_launcher(train_seq_model, num_processes=2)

Launching a training on 2 GPUs.


ValueError: cannot find context for 'fork'